# Part 3 - RNN and LSTM

## Import libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from torch.utils.data import DataLoader

from src import *

## Create dataloader

In [3]:
dataloader = CSVDataLoader('data/MSFT_2006-01-01_to_2018-01-01.csv')
X_train_1, X_test_1 = dataloader.split_by_year([2006, 2007], [2008])

training_dataset_1 = StockTorchTransDataset(X_train_1, zscore_params=None, history=5)
testing_dataset_1 = StockTorchTransDataset(X_test_1, zscore_params=training_dataset_1.get_zscore_params(), history=5)

training_loader_1 = DataLoader(training_dataset_1, batch_size=1, shuffle=False, drop_last=False)
testing_loader_1 = DataLoader(testing_dataset_1, batch_size=1, shuffle=False, drop_last=False)

for sample in training_loader_1:
    X, y = sample
    print("X:")
    print(X)
    print("y:")
    print(y)
    print("Shapes:")
    print(X.shape, y.shape)
    break

X:
tensor([[[-0.0460, -0.0533,  0.0023, -0.0441, -0.4800,  0.0000,  1.0000,
           0.0000,  0.0000,  0.0000,  1.0000,  0.0000,  0.0000,  0.0000,
           0.0000,  1.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
           0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.1596,  1.8483, -0.5587,  0.2843, -0.4800,  0.0000,  0.0000,
           1.0000,  0.0000,  0.0000,  1.0000,  0.0000,  0.0000,  0.0000,
           0.0000,  1.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
           0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.0825,  0.3441, -0.2458,  0.0064, -0.4800,  0.0000,  0.0000,
           0.0000,  1.0000,  0.0000,  1.0000,  0.0000,  0.0000,  0.0000,
           0.0000,  1.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
           0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
         [-0.3802, -1.2453,  1.3468, -0.2462, -0.4800,  0.0000,  0.0000,
           0.0000,  0.0000,  1.0000,  1.0000,  0.0000,  0.0000,  0.0000,